# 入站认证

AgentKit 网关支持 MCP 服务和 MCP 工具集，MCP 服务和 MCP 工具集都通过远程 MCP 服务的形式通过网络进行调用。如果这些服务不设置入站认证，那么知道服务地址的任何人都可以随意使用，这会造成服务不可用，数据泄漏以及财产损失。入站认证可以为 MCP 服务和 MCP 工具集提供防护，通过 API Key 或 OAuth2 JWT 防止未授权的用户非法访问 MCP 服务和 MCP 工具集。

在本教程中，我们将演示如何为 MCP 服务和 MCP 工具集增加 API Key 入站认证和 OAuth2 JWT 入站认证，你将学会，

1. 如何在 AgentKit 网关中为 MCP 服务配置 API Key 入站认证。
2. 如何在 AgentKit 网关中为 MCP 服务配置 OAuth2 JWT 入站认证。
3. 如何在 Agent Identity 中创建用户池和客户端。

在开始本教程之前，请确保你已经**正确配置了凭证**。

> **注意**：如果你还没有配置凭证，请先学习 [`01-tutorials/01-runtime/00_configuration_and_credentials.ipynb`](../01-runtime/00_configuration_and_credentials.ipynb) 教程完成配置。

## 1. 为 MCP 服务配置 API Key 入站认证

在创建 MCP 服务或 MCP 工具集时，AgentKit 网关要求选择 API Key 或 OAuth2 JWT 形式的入站认证。API Key 是一种用于身份验证和授权的密钥。当 MCP 客户端携带唯一的 API Key 发起请求时，AgentKit 网关将校验 API Key 的合法性以确认调用者身份，并判断是否允许访问的简易且高效的认证方式。AgentKit 网关约定所有的 API Key 均通过 Authorization 请求头携带。每个 MCP 服务或 MCP 工具集支持最多 25 个 API Key，你可以为不同的用户分配不同的 API Key。API Key 长期有效，但我们建议你定时轮转 API Key，防止 API Key 泄漏造成的损失。

在 AgentKit 网关的控制台中创建 MCP 服务或 MCP 工具集时，AgentKit 网关默认使用 API Key 作为入站认证方式。在使用 AgentKit SDK 创建 MCP 服务或 MCP 工具集时，你也可以简单的配置 API Key。所有的 API Key 都是由 AgentKit 平台生成的，你不能指定 API Key 的内容。

In [ ]:
from agentkit.sdk.mcp.client import AgentkitMCPClient
from agentkit.sdk.mcp.types import (
    CreateMCPServiceRequest,
    NetworkForCreateMCPService,
    InboundAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerKeyAuthForCreateMCPService,
    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService,
    BackendForCreateMCPService,
    BackendCustomMcpForCreateMCPService,
    BackendCustomMcpPublicPackageForCreateMCPService,
)
import json
import uuid

id = str(uuid.uuid4())[-8:]
mcp_config = {
    "mcpServers": {
        "sequential-thinking": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-sequential-thinking"],
        },
    },
}

client = AgentkitMCPClient()
request = CreateMCPServiceRequest(
    Name=f"mcp-service-{id}",
    ProtocolType="MCP",
    Path="/mcp",
    NetworkConfiguration=NetworkForCreateMCPService(
        EnablePublicNetwork=True,
        EnablePrivateNetwork=False,
    ),
    # 使用 API Key 入站认证保证服务安全。你可以创建多个 API Key 用于区分用户身份。
    InboundAuthorizerConfiguration=InboundAuthorizerForCreateMCPService(
        AuthorizerType="ApiKey",
        Authorizer=InboundAuthorizerAuthorizerForCreateMCPService(
            KeyAuth=InboundAuthorizerAuthorizerKeyAuthForCreateMCPService(
                ApiKeys=[
                    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService(
                        Name=f"mcp-service-api-key-{id}-1",
                    ),
                    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService(
                        Name=f"mcp-service-api-key-{id}-2",
                    ),
                    InboundAuthorizerAuthorizerKeyAuthApiKeysItemForCreateMCPService(
                        Name=f"mcp-service-api-key-{id}-3",
                    ),
                ],
            ),
        ),
    ),
    BackendType="CustomPublic",
    BackendConfiguration=BackendForCreateMCPService(
        CustomMcpConfiguration=BackendCustomMcpForCreateMCPService(
            PublicPackage=BackendCustomMcpPublicPackageForCreateMCPService(
                McpType="Remote",
                PackageManagerType="NPX",
                RawConfig=json.dumps(mcp_config),
            )
        ),
    ),
    ProjectName="default",
)

mcp_service_id = ""
try:
    response = client.create_mcp_service(request)
    mcp_service_id = response.mcp_service_id
    print(f"MCP service created successfully! MCP service ID: {mcp_service_id}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

In [ ]:
import time
from agentkit.sdk.mcp.types import (
    GetMCPServiceRequest,
)

request = GetMCPServiceRequest(
    MCPServiceId=mcp_service_id,
)

endpoint = ""
api_keys = []
try:
    while True:
        response = client.get_mcp_service(request)
        if response.mcp_service.status == "Ready":
            endpoint = response.mcp_service.network_configurations[0].endpoint
            for (
                api_key
            ) in (
                response.mcp_service.inbound_authorizer_configuration.authorizer.key_auth.api_keys
            ):
                api_keys.append(api_key.key)
            break
        if response.mcp_service.status != "Creating":
            print(f"The MCP service is in {response.mcp_service.status}")
            raise RuntimeError()
        time.sleep(2)
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

url = f"{endpoint}/mcp"
print(f"MCP service URL: {url}")
print(f"MCP service API key:")
for api_key in api_keys:
    print(f"- {api_key}")

恭喜！你已经使用 AgentKit 网关部署了具有 API Key 认证的 MCP 服务，这个 MCP 服务可以使用 3 个 API Key 进行访问。接下来，我们将尝试为 MCP 服务配置 OAuth2 JWT 入站认证。

## 2. 为 MCP 服务配置 OAuth2 JWT 入站认证

OAuth2 是一种授权协议，允许你授权第三方应用访问你在另一个服务上的受保护信息，而无需提供登录凭据。AgentKit Gateway 支持使用 Agent Identity 作为身份提供商（IdP），用户的账号、密码等敏感信息存储在 Agent Identity 中，MCP 客户端携带 Agent Identity 颁发的 JWT 短期访问令牌访问 AgentKit Gateway。AgentKit Gateway 将检查 Authorization 请求头中的访问令牌是否合法且在有效期内，以及颁发者（issuer）、客户端 ID、权限范围（scopes）和受众（audiences）是否和你预先设置的认证策略一致，并判断是否允许访问。相比于 API Key，OAuth2 JWT 有着权限控制等更高级的特性，也可以减少凭据泄漏的风险。

Agent Identity 是用户与智能体的身份认证与权限控制平台。我们需要首先创建身份池用于隔离不同系统间的身份管理，然后再在身份池内创建客户端用于标记不同的智能体。

首先，我们在 Agent Identity 中创建用户池和客户端。

> **注意**：创建 Agent Identity 用户池和客户端需要使用火山引擎 SDK。因此，你需要将 `VOLCENGINE_ACCESS_KEY`、`VOLCENGINE_SECRET_KEY` 和 `VOLCENGINE_REGION` 配置在 `.env` 文件中。

In [ ]:
!pip install volcengine-python-sdk --quiet

In [ ]:
import dotenv
import os
import uuid
import volcenginesdkcore
from volcenginesdkid import (
    IDApi,
    CreateUserPoolRequest,
    CreateUserPoolResponse,
    GetUserPoolRequest,
    GetUserPoolResponse,
    CreateUserPoolClientRequest,
    CreateUserPoolClientResponse,
)

dotenv.load_dotenv()

id = str(uuid.uuid4())[-8:]

configuration = volcenginesdkcore.Configuration()
configuration.ak = os.getenv("VOLCENGINE_ACCESS_KEY")
configuration.sk = os.getenv("VOLCENGINE_SECRET_KEY")
configuration.region = os.getenv("VOLCENGINE_REGION")
volcenginesdkcore.Configuration.set_default(configuration)

api_instance = IDApi()
request = CreateUserPoolRequest(
    name=f"user-pool-{id}",
    sign_in_attributes=["preferred_username"],
    required_sign_up_attributes=["preferred_username"],
    project_name="default",
)

user_pool_id = ""
try:
    response: CreateUserPoolResponse = api_instance.create_user_pool(request)
    user_pool_id = response.uid
    print(f"User pool created successfully!\n  User pool ID: {user_pool_id}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

request = GetUserPoolRequest(
    user_pool_uid=user_pool_id,
)

discovery_url = ""
try:
    response: GetUserPoolResponse = api_instance.get_user_pool(request)
    discovery_url = response.discovery_url
    print(f"  Discovery URL: {discovery_url}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e


request = CreateUserPoolClientRequest(
    user_pool_uid=user_pool_id,
    name=f"client-{id}",
    client_type="MACHINE_TO_MACHINE",
)

client_id = ""
client_secret = ""
try:
    response: CreateUserPoolClientResponse = api_instance.create_user_pool_client(
        request
    )
    client_id = response.uid
    client_secret = response.client_secret
    print(
        f"Client created successfully!\n  Client ID: {client_id}\n  Client secret: {client_secret}"
    )
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

AgentKit Gateway OAuth2 JWT 入站认证支持允许用户池中所有的客户端访问，也支持仅允许一部分客户端访问。如果需要将认证粒度控制在客户端，AgentKit Gateway 支持配置最多 5 个不同的客户端。

接下来，我们使用刚刚创建好的用户池，创建 MCP 服务并配置 OAuth2 JWT 入站认证。

In [ ]:
from agentkit.sdk.mcp.client import AgentkitMCPClient
from agentkit.sdk.mcp.types import (
    CreateMCPServiceRequest,
    NetworkForCreateMCPService,
    InboundAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerForCreateMCPService,
    InboundAuthorizerAuthorizerCustomJwtAuthorizerForCreateMCPService,
    BackendForCreateMCPService,
    BackendCustomMcpForCreateMCPService,
    BackendCustomMcpPublicPackageForCreateMCPService,
)
import json
import uuid

id = str(uuid.uuid4())[-8:]
mcp_config = {
    "mcpServers": {
        "sequential-thinking": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-sequential-thinking"],
        },
    },
}

client = AgentkitMCPClient()
request = CreateMCPServiceRequest(
    Name=f"mcp-service-{id}",
    ProtocolType="MCP",
    Path="/mcp",
    NetworkConfiguration=NetworkForCreateMCPService(
        EnablePublicNetwork=True,
        EnablePrivateNetwork=False,
    ),
    # 使用 OAuth2 JWT 入站认证保证服务安全。通过不配置 `AllowedClients`，放行用户池中所有的客户端。
    InboundAuthorizerConfiguration=InboundAuthorizerForCreateMCPService(
        AuthorizerType="CustomJWT",
        Authorizer=InboundAuthorizerAuthorizerForCreateMCPService(
            CustomJwtAuthorizer=InboundAuthorizerAuthorizerCustomJwtAuthorizerForCreateMCPService(
                DiscoveryUrl=discovery_url,
                AllowedClients=[],
            )
        ),
    ),
    BackendType="CustomPublic",
    BackendConfiguration=BackendForCreateMCPService(
        CustomMcpConfiguration=BackendCustomMcpForCreateMCPService(
            PublicPackage=BackendCustomMcpPublicPackageForCreateMCPService(
                McpType="Remote",
                PackageManagerType="NPX",
                RawConfig=json.dumps(mcp_config),
            )
        ),
    ),
    ProjectName="default",
)

mcp_service_id = ""
try:
    response = client.create_mcp_service(request)
    mcp_service_id = response.mcp_service_id
    print(f"MCP service created successfully! MCP service ID: {mcp_service_id}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

AgentKit Gatway 部署 MCP 服务需要一段时间。等到 MCP 服务部署完成后，我们通过 OAuth2 流程测试一下 MCP 服务。

In [ ]:
import time
from agentkit.sdk.mcp.types import (
    GetMCPServiceRequest,
)

request = GetMCPServiceRequest(
    MCPServiceId=mcp_service_id,
)

endpoint = ""
try:
    while True:
        response = client.get_mcp_service(request)
        if response.mcp_service.status == "Ready":
            endpoint = response.mcp_service.network_configurations[0].endpoint
            break
        if response.mcp_service.status != "Creating":
            print(f"The MCP service is in {response.mcp_service.status}")
            raise RuntimeError()
        time.sleep(2)
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

url = f"{endpoint}/mcp"
print(f"MCP service URL: {url}")

In [ ]:
!pip install fastmcp --quiet

In [ ]:
import base64
import requests

response = requests.request("GET", discovery_url)
token_endpoint = response.json()["token_endpoint"]

authorization = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

headers = {
    "Authorization": f"Basic {authorization}",
    "Content-Type": "application/x-www-form-urlencoded",
}
payload = "grant_type=client_credentials"

access_token = ""
try:
    response = requests.request(
        "POST",
        token_endpoint,
        headers=headers,
        data=payload,
    )
    access_token = response.json()["access_token"]
    print(f"Access token: {access_token}")
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

In [ ]:
from fastmcp import Client
from fastmcp.client.transports import StreamableHttpTransport

transport = StreamableHttpTransport(
    url=url,
    headers={
        "Authorization": f"Bearer {access_token}",
    }
)
mcp_client = Client(transport)

async with mcp_client:
    tools = await mcp_client.list_tools()
    print(f"The MCP server has a \"{tools[0].name}\" MCP tool.")

恭喜！你已经使用 AgentKit 网关部署了具有 OAuth2 JWT 认证的 MCP 服务，这个 MCP 服务可以使用 OAuth2 流程进行访问。

## 总结

恭喜你！现在你已经掌握了在 AgentKit 网关中为 MCP 服务配置入站认证的方法，

1. 为 MCP 服务配置 API Key 入站认证。
2. 为 MCP 服务配置 OAuth2 JWT 入站认证。

现在，请继续学习其他教程。